# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from config import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

609

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
# API set up
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + "809d895ef321b812b0cf144f02b8b8e5"

# Lists to hold data
c_id = []
c_name = []
country = []
lat = []
long = []
weather_date = []
temp = []
humidity = []
cloudiness = []
w_speed = []

# Create a file to store Open WeatherMap API call info and store as a .csv
owmlog = open('open_weathermap_log.txt', 'w')
owmdata = open('open_weathermap_data.csv', 'w')

# Define header for open_weathermap_data.csv
owmdata.write("c_id,c_name,country,lat,long,weather_date,temp,humidity,cloudiness,w_speed\n")

# Loop through the list of tuples of cities,country codes and perform a request for data on each
for city in cities:
    query_url = f"{url}&q={city}"
    response = requests.get(query_url).json()
    
# Try to querry the pertinent data, creating variables with proper formatting
    try:
        cityID = int(response['id'])
        cityName = response['name']
        countryCode = response['sys']['country']
        lat_val = response['coord']['lat']
        lon_val = response['coord']['lon']
        weatherDate = (datetime.fromtimestamp(response['dt'])).strftime('%m/%d/%y')
        temperature = response['main']['temp']
        humidity_val = response['main']['humidity']
        cloudiness_val = response['clouds']['all']
        windSpeed = response['wind']['speed']
        
        # Append retreived values to lists above using the variables created above
        c_id.append(cityID)
        c_name.append(cityName)
        country.append(countryCode)
        lat.append(lat_val)
        long.append(lon_val)
        weather_date.append(weatherDate)
        temp.append(temperature)
        humidity.append(humidity_val)
        cloudiness.append(cloudiness_val)
        w_speed.append(windSpeed)
       
        # Print cityname & cityID  to mark progress and store it to a print log text file
        print(f"{cityName}:{cityID} was found in the Open Weathermap API\n")
        owmlog.write(f"{cityName}:{cityID} was found in the Open Weathermap API\n")
        
        # Store the data as a .csv file
        owmdata.write(f"{cityID},{cityName},{countryCode},{lat_val},{lon_val},{weatherDate},\
            {temperature},{humidity_val},{cloudiness_val},{windSpeed}\n")
        
    # If the data is not in Open WeatherMap API and it creates a Null
    except:
        
        print(f"{city} was not found in the Open Weathermap API\n")
        owmlog.write(f"{city} was not found in the Open Weathermap API\n")
        
        #Add these cities to the lists
        c_id.append(None)
        c_name.append(None)
        country.append(None)
        lat.append(None)
        long.append(None)
        weather_date.append(None)
        temp.append(None)
        humidity.append(None)
        cloudiness.append(None)
        w_speed.append(None)
        pass
    
# Close the file handle after completion of the loop
owmlog.close()
owmdata.close()

umzimvubu was not found in the Open Weathermap API

geraldton was not found in the Open Weathermap API

jalu was not found in the Open Weathermap API

cabo san lucas was not found in the Open Weathermap API

lagoa was not found in the Open Weathermap API

vila velha was not found in the Open Weathermap API

buba was not found in the Open Weathermap API

rocha was not found in the Open Weathermap API

naryan-mar was not found in the Open Weathermap API

sieradz was not found in the Open Weathermap API

hithadhoo was not found in the Open Weathermap API

mataura was not found in the Open Weathermap API

mount gambier was not found in the Open Weathermap API

carnarvon was not found in the Open Weathermap API

kalyazin was not found in the Open Weathermap API

punta arenas was not found in the Open Weathermap API

pevek was not found in the Open Weathermap API

katsuura was not found in the Open Weathermap API

namatanai was not found in the Open Weathermap API

codrington was not found i

miraflores was not found in the Open Weathermap API

aklavik was not found in the Open Weathermap API

bengkulu was not found in the Open Weathermap API

saint anthony was not found in the Open Weathermap API

yulara was not found in the Open Weathermap API

coruripe was not found in the Open Weathermap API

ferme-neuve was not found in the Open Weathermap API

kirakira was not found in the Open Weathermap API

dicabisagan was not found in the Open Weathermap API

kingston was not found in the Open Weathermap API

alice springs was not found in the Open Weathermap API

lorengau was not found in the Open Weathermap API

oyama was not found in the Open Weathermap API

skjervoy was not found in the Open Weathermap API

walvis bay was not found in the Open Weathermap API

baihar was not found in the Open Weathermap API

bethel was not found in the Open Weathermap API

bracebridge was not found in the Open Weathermap API

kapaa was not found in the Open Weathermap API

pangnirtung was not f

gat was not found in the Open Weathermap API

alta floresta was not found in the Open Weathermap API

ostrovnoy was not found in the Open Weathermap API

navapur was not found in the Open Weathermap API

port shepstone was not found in the Open Weathermap API

bethlehem was not found in the Open Weathermap API

springdale was not found in the Open Weathermap API

hauterive was not found in the Open Weathermap API

karratha was not found in the Open Weathermap API

mys shmidta was not found in the Open Weathermap API

clyde river was not found in the Open Weathermap API

tevaitoa was not found in the Open Weathermap API

half moon bay was not found in the Open Weathermap API

cartagena was not found in the Open Weathermap API

olafsvik was not found in the Open Weathermap API

leh was not found in the Open Weathermap API

ishurdi was not found in the Open Weathermap API

bandarbeyla was not found in the Open Weathermap API

freetown was not found in the Open Weathermap API

beitbridge w

tairua was not found in the Open Weathermap API

carros was not found in the Open Weathermap API

inverness was not found in the Open Weathermap API

daitari was not found in the Open Weathermap API

thongwa was not found in the Open Weathermap API

diamantino was not found in the Open Weathermap API

lydenburg was not found in the Open Weathermap API

naron was not found in the Open Weathermap API

mitu was not found in the Open Weathermap API

uusikaupunki was not found in the Open Weathermap API

palmer was not found in the Open Weathermap API

loikaw was not found in the Open Weathermap API

newport was not found in the Open Weathermap API

hamilton was not found in the Open Weathermap API

kudahuvadhoo was not found in the Open Weathermap API

basco was not found in the Open Weathermap API

yar-sale was not found in the Open Weathermap API

partapur was not found in the Open Weathermap API

nyurba was not found in the Open Weathermap API

san quintin was not found in the Open Weat

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression